In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))


In [2]:
import os
from pathlib import Path

import paramiko
from scp import SCPClient

import getpass
import gc

/media/rich/OS/Users/Richard/Linux_stuff_on_OS/conda_envs/envs/suite2p/lib/python3.8/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [179]:
dir_github = '/media/rich/Home_Linux_partition/github_repos/'
import sys
sys.path.append(dir_github)

%load_ext autoreload
%autoreload 2
from s2p_on_o2 import util

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# paramiko.util.log_to_file("paramiko.log")

----
# ==== Optional ====
### transfer raw data from local machine to `MICROSCOPE`

In [86]:
## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to
dir_data_local = '/media/rich/bigSSD/NMA_data'
dir_data_remote = '/n/data1/hms/neurobio/sabatini/rich/data/test2'  ## contents of dir_data_local will be dumped into dir_data_remote

print(f'contents of    {dir_data_local}    will be copied to    {dir_data_remote}')

contents of    /media/rich/bigSSD/NMA_data    will be copied to    /n/data1/hms/neurobio/sabatini/rich/data/test2    using fxp protocol


In [57]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [129]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

sftp.put_dir(dir_data_local, dir_data_remote)

------------
# ==== Start ====

## Prepare directories

In [5]:
dir_S2pOutput_remote = '/n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/testRun'  ## outer directory. Final outputs will be in an inner directory
# folderName_batch_save = 'job_'  ## name of inner folder that will be created


path_dispatcher_local = Path(r'/media/rich/Home_Linux_partition/github_repos/s2p_on_o2/').resolve() / 'dispatcher.py'  ## path to the dispatcher.py file on local computer
path_dispatcher_remote = Path(r'/n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/') / 'dispatcher.py'  ## path to where you want the dispatcher.py file to be copied onto the server
path_s2pScript_remote = '/n/data1/hms/neurobio/sabatini/rich/github_repos/s2p_on_o2/remote_run_s2p.py'  ## path to a copy of the remote_run_s2p.py file within the s2p_on_o2 repo on the server

## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to
dir_data_local = '/n/files/Neurobio/MICROSCOPE/Rich/data/test1/'
dir_data_remote = '/n/data1/hms/neurobio/sabatini/rich/data/test'  ## contents of dir_data_local will be dumped into dir_data_remote

mode_transfer = 'fxp'  ## either ftp (from local machine to server) or fxp (server to server)
print(f'contents of    {dir_data_local}    will be copied to    {dir_data_remote}    using {mode_transfer} protocol')

contents of    /n/files/Neurobio/MICROSCOPE/Rich/data/test1/    will be copied to    /n/data1/hms/neurobio/sabatini/rich/data/test    using fxp protocol


## Prepare username + password

In [7]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [ ]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

In [9]:
## initialize ssh_transfer
ssh_t = util.ssh_interface(
    nbytes_toReceive=4096,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
)

In [ ]:
## initialize ssh_compute
ssh_c = util.ssh_interface(
    nbytes_toReceive=4096,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
)

## Prepare commands

In [11]:
prompt_snip = f'[{username}'

commands = {
    'make_dir': f"mkdir {dir_data_remote}",
    'copy_data': f"cp -r {str(Path(dir_data_local).resolve())}/. {dir_data_remote}",
    'dispatch_s2p': f"python {str(path_dispatcher_remote)} {dir_S2pOutput_remote} {path_s2pScript_remote} {dir_data_remote}"  ## dispatcher expecting these args as inputs
}
display(commands)

{'make_dir': 'mkdir /n/data1/hms/neurobio/sabatini/rich/data/test',
 'copy_data': 'cp -r /n/files/Neurobio/MICROSCOPE/Rich/data/test1/. /n/data1/hms/neurobio/sabatini/rich/data/test',
 'dispatch_s2p': 'python /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/testRun /n/data1/hms/neurobio/sabatini/rich/github_repos/s2p_on_o2/remote_run_s2p.py /n/data1/hms/neurobio/sabatini/rich/data/test'}

## Upload `dispatcher.py` file

In [36]:
sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote))

<SFTPAttributes: [ size=4689 uid=140162 gid=3416 mode=0o100664 atime=1652787167 mtime=1652789733 ]>

## Copy data from `MICROSCOPE` to `data1`

In [12]:
ssh_t.send_receive(commands['make_dir']);

t
mkdir: cannot create directory ‘/n/data1/hms/neurobio/sabatini/rich/data/test’: File exists
(base) [rh183@transfer08 ~]$ 


In [13]:
ssh_t.send(commands['copy_data'])
ssh_t.expect(str_success=prompt_snip, total_timeout=120, verbose=True);

. /n/data1/hms/neurobio/sabatini/rich/data/testst1/ 

(base) [rh183@transfer08 ~]$ 


## Dispatch `remote_run_s2p.py`

In [38]:
ssh_c.send(commands['dispatch_s2p']);

ssh_c.expect(str_success=prompt_snip);

_run_s2p.py /n/data1/hms/neurobio/sabatini/rich/data/test_repos/s2p_on_o2/remote 

/n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/testRun /n/data1/hms/neurobio/sabatini/rich/github_repos/s2p_on_o2/remote_run_s2p.py /n/data1/hms/neurobio/sabatini/rich/data/test

Submitting job: jobNum_ 0

Submitted batch job 53400568

(base) [rh183@login04 ~]$ 


## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs back from `data1` to `MICROSCOPE`

## Prepare username + password

In [7]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `ssh` object

In [48]:
## initialize ssh_transfer
ssh_t = util.ssh_interface(
    nbytes_toReceive=4096,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
)

Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...
Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on transfer08 which is a
4 core system with 7.62 GiB memory
running kernel 3.10.0 born on 2021-06-23

==== O2 ===================================================================

  News (Jan 3 2022)
  

  +-------------------------------------------------------------------------------+
  | ##################### RC Core launched in July 2021 ######################### |
  |                                                                               |
  | Since July 2021, researchers whose PI does not have a primary or secondary    |
  | faculty appointment in an HMS Basic or Socia

In [140]:
dir_S2pOutput_remote = dir_S2pOutput_remote  ## from above
dir_s2pOutput_MICROSCOPE = '/n/files/Neurobio/MICROSCOPE/Rich/data/test1'
print(f'contents of    {dir_data_remote}    will be copied to    {dir_s2pOutput_local}')

contents of    /n/data1/hms/neurobio/sabatini/rich/data/test2    will be copied to    /media/rich/bigSSD/tmp_data/test2


In [50]:
commands = {
    'make_dir': f"mkdir {dir_s2pOutput_MICROSCOPE}",
    'copy_s2p': f"cp -r {str(Path(dir_S2pOutput_remote).resolve())}/. {dir_s2pOutput_MICROSCOPE}",
}
display(commands)

{'make_dir': 'mkdir /n/files/Neurobio/MICROSCOPE/Rich/data/2pRAM/face_rhythm_motor_mapping/test/',
 'copy_s2p': 'cp -r /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/testRun/. /n/files/Neurobio/MICROSCOPE/Rich/data/2pRAM/face_rhythm_motor_mapping/test/'}

In [51]:
ssh_t.send_receive(commands['make_dir']);

face_rhythm_motor_mapping/test/
(base) [rh183@transfer08 ~]$ 


In [54]:
ssh_t.send(commands['copy_s2p'])
ssh_t.expect(str_success=prompt_snip, total_timeout=120, verbose=True);

m_motor_mapping/test/n/. /n/files/Neurobio/MICROSCOPE/Rich/data/2pRAM/face_rhyth 

(base) [rh183@transfer08 ~]$ 


## Clean up

In [ ]:
## delete password
del pw
gc.collect()
    
## close ssh
ssh_t.close()
del ssh_t
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs from `MICROSCOPE` to local machine

In [148]:
dir_s2pOutput_MICROSCOPE = dir_s2pOutput_MICROSCOPE  ## from above
dir_s2pOutput_local = '/media/rich/bigSSD/tmp_data/test2'
print(f'contents of    {dir_s2pOutput_MICROSCOPE}    will be copied to    {dir_s2pOutput_local}')

contents of    /n/files/Neurobio/MICROSCOPE/Rich/data/test1    will be copied to    /media/rich/bigSSD/tmp_data/test2


In [57]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [ ]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

In [185]:
sftp.get_dir(dir_s2pOutput_MICROSCOPE, dir_s2pOutput_local)

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp